<a href="https://colab.research.google.com/github/aaroncantu22/CSCI164AIProjects/blob/main/AI24Ch3a_Modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modified AI Tile Puzzle Solver: 3x3 and 4x4

In [1]:

import random
import heapq
import time

class Puzzle:
    def __init__(self, dim):
        self.dim = dim
        self.goal = list(range(1, dim*dim)) + [0]

    def actions(self, state):
        i = state.index(0)
        row, col = divmod(i, self.dim)
        moves = []
        if row > 0: moves.append('u')
        if row < self.dim - 1: moves.append('d')
        if col > 0: moves.append('l')
        if col < self.dim - 1: moves.append('r')
        return moves

    def result(self, state, action):
        i = state.index(0)
        row, col = divmod(i, self.dim)
        if action == 'u': j = (row-1)*self.dim + col
        if action == 'd': j = (row+1)*self.dim + col
        if action == 'l': j = row*self.dim + (col-1)
        if action == 'r': j = row*self.dim + (col+1)
        new_state = state[:]
        new_state[i], new_state[j] = new_state[j], new_state[i]
        return new_state

    def goal_test(self, state):
        return state == self.goal

    def print_state(self, state):
        for i in range(self.dim):
            print(state[i*self.dim:(i+1)*self.dim])


In [2]:

def random_walk(puzzle, steps):
    state = puzzle.goal[:]
    last_action = None
    for _ in range(steps):
        possible = puzzle.actions(state)
        if last_action:
            possible = [a for a in possible if a != {'u':'d','d':'u','l':'r','r':'l'}[last_action]]
        action = random.choice(possible)
        state = puzzle.result(state, action)
        last_action = action
    return state


In [3]:

def misplaced_tiles(state, goal):
    return sum([1 for i in range(len(state)) if state[i] != 0 and state[i] != goal[i]])

def manhattan_distance(state, goal, dim):
    distance = 0
    for i in range(1, dim*dim):
        xi, yi = divmod(state.index(i), dim)
        xg, yg = divmod(goal.index(i), dim)
        distance += abs(xi - xg) + abs(yi - yg)
    return distance


In [4]:

def bfs(puzzle, start):
    frontier = [[start]]
    explored = set()
    nodes_expanded = 0
    while frontier:
        path = frontier.pop(0)
        state = path[-1]
        if puzzle.goal_test(state):
            return path, nodes_expanded
        explored.add(tuple(state))
        for action in puzzle.actions(state):
            child = puzzle.result(state, action)
            if tuple(child) not in explored:
                frontier.append(path + [child])
        nodes_expanded += 1
    return None, nodes_expanded

def astar(puzzle, start, heuristic):
    frontier = []
    heapq.heappush(frontier, (0, [start]))
    explored = set()
    nodes_expanded = 0
    while frontier:
        _, path = heapq.heappop(frontier)
        state = path[-1]
        if puzzle.goal_test(state):
            return path, nodes_expanded
        explored.add(tuple(state))
        for action in puzzle.actions(state):
            child = puzzle.result(state, action)
            if tuple(child) not in explored:
                g = len(path)
                if heuristic.__name__ == "manhattan_distance":
                    h = heuristic(child, puzzle.goal, puzzle.dim)
                else:
                    h = heuristic(child, puzzle.goal)
                f = g + h
                heapq.heappush(frontier, (f, path + [child]))
        nodes_expanded += 1
    return None, nodes_expanded


In [5]:

def run_experiments():
    sizes = [3, 4]
    depths = [5, 10, 20, 40, 80]
    for size in sizes:
        puzzle = Puzzle(size)
        print(f"\nRunning tests for {size}x{size} puzzle...")
        for depth in depths:
            print(f"\nRandom Walk Depth: {depth}")
            for trial in range(3):
                start = random_walk(puzzle, depth)
                print(f"\nTrial {trial+1}")
                puzzle.print_state(start)
                print("BFS:")
                path, expanded = bfs(puzzle, start)
                print(f"Solution length: {len(path)-1 if path else 'N/A'} | Nodes expanded: {expanded}")
                print("A* Misplaced Tiles:")
                path, expanded = astar(puzzle, start, misplaced_tiles)
                print(f"Solution length: {len(path)-1 if path else 'N/A'} | Nodes expanded: {expanded}")
                print("A* Manhattan:")
                path, expanded = astar(puzzle, start, manhattan_distance)
                print(f"Solution length: {len(path)-1 if path else 'N/A'} | Nodes expanded: {expanded}")


In [7]:
run_experiments()


Running tests for 3x3 puzzle...

Random Walk Depth: 5

Trial 1
[1, 5, 2]
[0, 4, 3]
[7, 8, 6]
BFS:
Solution length: 5 | Nodes expanded: 51
A* Misplaced Tiles:
Solution length: 5 | Nodes expanded: 5
A* Manhattan:
Solution length: 5 | Nodes expanded: 5

Trial 2
[1, 2, 3]
[0, 8, 5]
[4, 7, 6]
BFS:
Solution length: 5 | Nodes expanded: 46
A* Misplaced Tiles:
Solution length: 5 | Nodes expanded: 5
A* Manhattan:
Solution length: 5 | Nodes expanded: 5

Trial 3
[1, 2, 3]
[0, 8, 5]
[4, 7, 6]
BFS:
Solution length: 5 | Nodes expanded: 46
A* Misplaced Tiles:
Solution length: 5 | Nodes expanded: 5
A* Manhattan:
Solution length: 5 | Nodes expanded: 5

Random Walk Depth: 10

Trial 1
[0, 2, 3]
[1, 6, 8]
[4, 7, 5]
BFS:
Solution length: 8 | Nodes expanded: 173
A* Misplaced Tiles:
Solution length: 8 | Nodes expanded: 11
A* Manhattan:
Solution length: 8 | Nodes expanded: 8

Trial 2
[0, 3, 6]
[1, 4, 8]
[7, 2, 5]
BFS:
Solution length: 10 | Nodes expanded: 559
A* Misplaced Tiles:
Solution length: 10 | Nodes ex

KeyboardInterrupt: 